In [3]:
import pandas as pd
import re
import os

In [5]:
# Check to see if current job posting file exists to combine with cumulative data
cumulative_job_posting_df = pd.read_csv("../Data/Job_posting_data_cumulative.csv")

file_path = '../Data/Job_posting_data_current.csv'
flag = os.path.isfile(file_path)

if flag:
    #open the cumulative CSV file as a DataFrame
    new_job_posting_df = pd.read_csv(file_path)
    #append the DataFrames together
    total_job_posting_df = pd.concat([cumulative_job_posting_df, new_job_posting_df])
else:
    #If the current file doesn't exist, we just use the cumulative file
    total_job_posting_df = cumulative_job_posting_df

# Show the new cumulative data set
print(total_job_posting_df)

                                              Job Title  \
0                            route sales representative   
1                                 ultrasound technician   
2                      optometric ophthalmic technician   
3                                            controller   
4                                       account manager   
...                                                 ...   
8646  san diego – cannabis outside sales representative   
8647                                      server runner   
8648  aircraft maintenance technician, line maintenance   
8649                             retail store associate   
8650                           manufacturing technician   

                   Company Name  \
0       frito lay north america   
1         proximity diagnostics   
2      imperial beach optometry   
3                      filtrous   
4                 aya corporate   
...                         ...   
8646  calyx brands distribution   
8647       

In [6]:
# Drop rows with blank "job title" or "company name"

total_job_posting_df.dropna(subset=['Job Title', 'Company Name'], inplace=True)

In [7]:
# Format the "job title" and "company name" columns
total_job_posting_df['Job Title'] = total_job_posting_df['Job Title'].str.replace('/', ' ')
total_job_posting_df['Job Title'] = total_job_posting_df['Job Title'].str.replace('-', ' ')
total_job_posting_df['Job Title'] = total_job_posting_df['Job Title'].str.lower()

total_job_posting_df['Company Name'] = total_job_posting_df['Company Name'].str.replace('/', ' ')
total_job_posting_df['Company Name'] = total_job_posting_df['Company Name'].str.replace('-', ' ')
total_job_posting_df['Company Name'] = total_job_posting_df['Company Name'].str.lower()
print(total_job_posting_df)

                                              Job Title  \
0                            route sales representative   
1                                 ultrasound technician   
2                      optometric ophthalmic technician   
3                                            controller   
4                                       account manager   
...                                                 ...   
8646  san diego – cannabis outside sales representative   
8647                                      server runner   
8648  aircraft maintenance technician, line maintenance   
8649                             retail store associate   
8650                           manufacturing technician   

                   Company Name  \
0       frito lay north america   
1         proximity diagnostics   
2      imperial beach optometry   
3                      filtrous   
4                 aya corporate   
...                         ...   
8646  calyx brands distribution   
8647       

# Here, we remove the duplicate rows, (ignoring the "Date Posted" b/c the same job can be posted on multiple days)


In [8]:
#Find duplicate job postings
total_job_posting_df.loc[total_job_posting_df.duplicated(subset=['Job Title', 'Company Name', 'Location'])]

,Job Title,Company Name,Location,Remote,Salary,Full Time,Part Time,Date Posted


In [9]:
# Remove duplicate job postings
total_job_posting_df.drop_duplicates(subset=['Job Title', 'Company Name', 'Location', 'Remote', 'Salary', 'Full Time', 'Part Time'],inplace=True, keep='last')
print(str(len(total_job_posting_df)) + " unique job postings")

8651 unique job postings


In [10]:
# Save the new cumulative data set
total_job_posting_df.to_csv('../Data/Job_posting_data_cumulative.csv', index=False)


In [11]:
job_salaries = total_job_posting_df['Salary']
print(job_salaries)

0                    $1,179 a week
1           $1,800 - $3,200 a week
2                $10 - $18 an hour
3       $100,000 - $150,000 a year
4       $100,000 - $175,000 a year
                   ...            
8646     $50,000 - $150,000 a year
8647                           NaN
8648                           NaN
8649              From $15 an hour
8650             Up to $27 an hour
Name: Salary, Length: 8651, dtype: object


In [12]:
total_job_posting_df['Salary Type'] = total_job_posting_df['Salary'].str.split().str[-1]
display(total_job_posting_df['Salary Type'].unique())

array(['week', 'hour', 'year', 'month', 'day', nan, 'mile', 'session'],
      dtype=object)

In [13]:
# First, remove commas from Salary
total_job_posting_df['Salary'] = total_job_posting_df['Salary'].str.replace(',','')
print(total_job_posting_df)

                                              Job Title  \
0                            route sales representative   
1                                 ultrasound technician   
2                      optometric ophthalmic technician   
3                                            controller   
4                                       account manager   
...                                                 ...   
8646  san diego – cannabis outside sales representative   
8647                                      server runner   
8648  aircraft maintenance technician, line maintenance   
8649                             retail store associate   
8650                           manufacturing technician   

                   Company Name  \
0       frito lay north america   
1         proximity diagnostics   
2      imperial beach optometry   
3                      filtrous   
4                 aya corporate   
...                         ...   
8646  calyx brands distribution   
8647       

In [14]:
def get_salary_raw_number(row):
    # See if the salary data is a string
    if isinstance(row, str):
        # Find the word "Estimated"
        estimated = row.find("Estimated")
        if estimated != -1:
            # Find the raw number for salary $##.#K
            raw_number = re.search('(\d+(\.\d{1,2})?)K', row).group()
            raw_number = raw_number[:-1]
            raw_number = 1000 * float(raw_number)
        else:
            raw_number = re.search('(\d+)', row).group()
            raw_number = float(raw_number)
    else:
        return None
    return raw_number

In [15]:
# Get salary number to compute yearly salary

total_job_posting_df['Salary (Raw Number)'] = total_job_posting_df['Salary'].apply(lambda x: get_salary_raw_number(x))
print(total_job_posting_df)

                                              Job Title  \
0                            route sales representative   
1                                 ultrasound technician   
2                      optometric ophthalmic technician   
3                                            controller   
4                                       account manager   
...                                                 ...   
8646  san diego – cannabis outside sales representative   
8647                                      server runner   
8648  aircraft maintenance technician, line maintenance   
8649                             retail store associate   
8650                           manufacturing technician   

                   Company Name  \
0       frito lay north america   
1         proximity diagnostics   
2      imperial beach optometry   
3                      filtrous   
4                 aya corporate   
...                         ...   
8646  calyx brands distribution   
8647       

In [16]:
def compute_yearly_salary(x):
    yearly_salary = None
    if x['Salary Type'] == "hour":
        yearly_salary = 2087 * int(x['Salary (Raw Number)'])
    elif x['Salary Type'] == "day":
        yearly_salary = 261 * int(x['Salary (Raw Number)'])
    elif x['Salary Type'] == "week":
        yearly_salary = 52 * int(x['Salary (Raw Number)'])
    elif x['Salary Type'] == "month":
        yearly_salary = 12 * int(x['Salary (Raw Number)'])
    elif x['Salary Type'] == "year":
        yearly_salary = int(x['Salary (Raw Number)'])
    else:
        yearly_salary = None
    return yearly_salary
        

In [17]:
total_job_posting_df['Annual Salary'] = total_job_posting_df.apply(compute_yearly_salary, axis=1)

# TODO: Format the Annual Salary column to currency
#job_posting_df['Annual Salary'] = job_posting_df['Annual Salary'].format("$*:")
display(total_job_posting_df)
                                                      

,Job Title,Company Name,Location,Remote,Salary,Full Time,Part Time,Date Posted,Salary Type,Salary (Raw Number),Annual Salary
0,route sales representative,frito lay north america,"San Diego, CA 92123 \n(Kearny Mesa area)\n+1 l...",False,$1179 a week,True,False,1/7/2022,week,1179.0,61308.0
1,ultrasound technician,proximity diagnostics,"San Diego, CA",False,$1800 - $3200 a week,False,True,1/7/2022,week,1800.0,93600.0
2,optometric ophthalmic technician,imperial beach optometry,"Imperial Beach, CA 91932\n+1 location",False,$10 - $18 an hour,True,False,1/7/2022,hour,10.0,20870.0
3,controller,filtrous,"Poway, CA 92064",False,$100000 - $150000 a year,True,False,1/7/2022,year,100000.0,100000.0
4,account manager,aya corporate,"San Diego, CA\n+18 locations",False,$100000 - $175000 a year,False,False,1/7/2022,year,100000.0,100000.0
...,...,...,...,...,...,...,...,...,...,...,...
8646,san diego – cannabis outside sales representative,calyx brands distribution,"Remote in San Diego, CA",True,$50000 - $150000 a year,True,False,2022-02-15,year,50000.0,50000.0
8647,server runner,chuck e cheese,"National City, CA 91950\n(Central area)\n+4 lo...",False,NaN,False,False,2022-02-15,NaN,NaN,NaN
8648,"aircraft maintenance technician, line maintenance",delta,"San Diego, CA",False,NaN,True,False,2022-02-15,NaN,NaN,NaN
8649,retail store associate,lamps plus,"San Diego, CA 92110\n(Morena area)",False,From $15 an hour,True,False,2022-02-15,hour,15.0,31305.0


In [18]:
# Drop the unneccessary column
total_job_posting_df.pop('Salary (Raw Number)')

0         1179.0
1         1800.0
2           10.0
3       100000.0
4       100000.0
          ...   
8646     50000.0
8647         NaN
8648         NaN
8649        15.0
8650        27.0
Name: Salary (Raw Number), Length: 8651, dtype: float64

In [19]:
# Save the .csv file
total_job_posting_df.to_csv('../Data/Job_Posting_Data_With_Annual_Salary.csv', index=False)